In [1]:
from typing import Tuple
import math

def normal_appromixation_to_binomial(n: int, p:float) -> Tuple[float,float]:
    """ Returns mu & sigma corresponding to a binomial(n,p) """

    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

# Normal Probability Above/Below/Between

In [2]:
from Helper.Probabilty import normal_cdf

# The normal cdf _is_ the probability thae variable is below a threshold 

normal_probability_below = normal_cdf

# It's above the threshold if it's not below the threshold
def normal_probability_above( lo: float,
                              mu: float = 0,
                              sigma: float = 1) -> float:
    """  The probabilty that an N(mu, sigma) is greater than lo."""
    return 1 - normal_cdf(lo, mu, sigma)

# if it's less than hi, but not less than lo
def normal_probability_between(lo: float,
                               hi: float,
                               mu: float,
                               sigma: float = 1) -> float:
    """ The probability that an N(mu, sigma) is between lo and hi """
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# It's outside if it's not between
def normal_probability_outside(lo: float,
                               hi: float,
                               mu: float,
                               sigma: float = 1) -> float:
    """ The probability that an N(mu, sigma) is not between lo and hi  """
    return 1 - normal_probability_between(lo, hi, mu, sigma)


# Inverse Probability by bounds 
 

In [3]:
from Helper.Probabilty import inverse_normal_cdf

def  normal_upper_bound(probability: float,
                        mu : float = 0,
                        sigma: float = 1) -> float:
    """ Returns the z for which P(Z >= z) = probability """
    return inverse_normal_cdf(probability, mu, sigma)

def  normal_lower_bound(probability: float,
                        mu : float = 0,
                        sigma: float = 1) -> float:
    """ Returns the z for which P(Z >= z) = probability """
    return inverse_normal_cdf( 1 - probability, mu, sigma)

def normal_two_sided_bounds(probability: float,
                            mu: float, 
                            sigma: float = 1) -> Tuple[float, float]:
    """ 
    Returns the symmetric (above the mean ) bounds
    that contains the specified probability
    """

    tail_probability = ( 1 - probability) / 2

    # Upper bound should have tail_probability above it
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)

    # Lower bound should have tail_probability below it
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)

    return lower_bound, upper_bound



In [4]:
mu_0, sigma_0 = normal_appromixation_to_binomial(1000, 0.5)
print(f' mu_0 {mu_0}  sigma {sigma_0}')

 mu_0 500.0  sigma 15.811388300841896


In [5]:
import math

lower_bounds, upper_bounds = normal_two_sided_bounds(0.95, mu_0, sigma_0)
print(f' Lower_bound = {lower_bounds}  Upper_Bounds = {upper_bounds}')

 Lower_bound = 469.01026640487555  Upper_Bounds = 530.9897335951244


In [6]:
# 95% bounds based on assumption p is 0.5
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

# actual mu & sigma based  on p = 0.55
mu_1, sigma_1 = normal_appromixation_to_binomial(1000, 0.55)

# a type 2 error means we fail to reject the null hyppothesis,
# which will happen when X is still in our Orignal interval
type_2_probaility = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probaility
print(power)

0.886548001295367


# p-Values

In [8]:
def two_sided_p_value(x: float, mu:float = 0, sigma: float = 1) -> float:
    """ 
    How likely are we to see a value at least as extreme as x (in either direction)
    if our values are from an N(mu, sigma) ?
    """

    if x >= mu:
        # x is greater than mean, so the tail is everything greater than x
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # x is less than the mean, so the tail is everything less than x
        return 2 * normal_probability_below(x, mu, sigma)

two_sided_p_value(529.5, mu_0, sigma_0)

0.06207721579598835

In [11]:
import random

extreme_value_count = 0
for _ in range(1000):
    num_heads = sum(1 if random.random() < 0.5 else 0
                    for _ in range (1000))
    if num_heads >= 530 or num_heads <=470:
        extreme_value_count +=1
    
extreme_value_count

60

# Confidence Intervals 

In [14]:
p = .525
math.sqrt( p * ( 1 - p) / 1000)

p_hat = 525 / 1000
mu = p_hat
sigma = math.sqrt( p_hat * (1 - p_hat) / 1000)
sigma

0.015791611697353755

# p-Hacking

In [15]:
from typing import List

def run_experiment() -> List[bool] :
    """ 
    Flips a fair coin 1000 times, Trye = heads, False = tails
    """
    return [random.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment: List[bool]) -> bool:
    """ 
    Using the 5% significance levels 
    """

    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

random.seed(0)
experiments = [ run_experiment() for _ in range(1000)]
num_rejections = len([experiment for experiment in experiments if reject_fairness(experiment)])

assert num_rejections ==46

# A/B Testing

In [17]:
def estimated_parameters(N: int, n: int) -> Tuple[float, float]:
    p = n / N
    sigma = math.sqrt(p * (1-p) / N)
    return p,sigma

def a_b_test_statistic( N_A:int, n_a: int, N_B: int, n_b: int) -> float:
    p_A, sigma_A =  estimated_parameters(N_A, n_a)
    p_B, sigma_B = estimated_parameters(N_B, n_b)
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

z = a_b_test_statistic(1000, 200, 1000, 180)
print(f' z value {z}')
print(f' two sided value {two_sided_p_value(z)}')

 z value -1.1403464899034472
 two sided value 0.2541419765422359
